In [0]:
# 引入必需的函数库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [0]:
# 划分数据
all_pd = pd.read_csv( "data_train.csv", sep=',', names=['K1K2驱动信号', '电子锁驱动信号','急停信号','门禁信号','THDV-M','THDI-M','label'])

In [6]:
# 探索性数据分析
all_pd.describe()

,K1K2驱动信号,电子锁驱动信号,急停信号,门禁信号,THDV-M,THDI-M,label
count,85500.000000,85500.000000,85500.000000,85500.000000,85500.000000,85500.000000,85500.000000
mean,11.809984,12.128766,-0.073558,12.060010,26.617066,37.528266,0.500000
std,0.015879,0.014637,0.014417,0.012433,53.560158,55.491146,0.500003
min,11.735606,12.056209,-0.141751,12.001741,8.455756,8.004304,0.000000
25%,11.801417,12.121097,-0.081192,12.053098,11.210131,18.775064,0.000000
50%,11.810531,12.129028,-0.073402,12.059551,15.576817,26.210275,0.500000
75%,11.818357,12.136946,-0.065624,12.067110,27.886568,40.446093,1.000000
max,11.880811,12.199404,-0.001241,12.114959,6861.198222,7197.115193,1.000000


In [7]:
# 加载测试数据
test_pd = pd.read_csv( "data_test.csv", sep=',', names=['K1K2驱动信号', '电子锁驱动信号','急停信号','门禁信号','THDV-M','THDI-M'])
test_pd.describe()

,K1K2驱动信号,电子锁驱动信号,急停信号,门禁信号,THDV-M,THDI-M
count,36644.000000,36644.000000,36644.000000,36644.000000,36644.000000,36644.000000
mean,11.810068,12.128750,-0.073496,12.059812,26.825217,40.186763
std,0.015947,0.014513,0.014525,0.012521,57.362011,370.765413
min,11.734548,12.059299,-0.137070,12.000966,8.819448,7.827390
25%,11.801417,12.121097,-0.081192,12.053098,11.246829,18.861033
50%,11.810531,12.129028,-0.073402,12.059551,15.638636,26.322794
75%,11.818357,12.136946,-0.065624,12.067110,28.105188,40.686875
max,11.889563,12.200959,-0.008032,12.111794,4838.301181,67911.581370


In [0]:
# pandas -> nparray，随机划分train/val，80：20
X = np.array(all_pd.iloc[:,0:-1].values.tolist(), dtype='float32')
y = np.array(all_pd.iloc[:,-1].values.tolist(), dtype='float32')

train_X,val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [30]:
# 决策树
from sklearn.tree import DecisionTreeClassifier
cls=DecisionTreeClassifier(max_depth=6,min_samples_split=10,min_samples_leaf=5,max_features=0.7)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

0.8946045478459109

In [37]:
# 随机森林
from sklearn.ensemble import RandomForestClassifier
cls=RandomForestClassifier(max_depth=100,min_samples_split=10,min_samples_leaf=5,max_features=0.8)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

0.9489496537965028

In [40]:
# 不知道什么树
from sklearn.ensemble import ExtraTreesClassifier
cls=ExtraTreesClassifier(max_depth=100,min_samples_split=10,min_samples_leaf=5,max_features=0.8)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

0.9636963696369636

In [44]:
# 梯度上升机
from sklearn.ensemble import GradientBoostingClassifier
cls=GradientBoostingClassifier(max_depth=10,min_samples_split=10,min_samples_leaf=5,max_features=0.7)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

0.994256915142991

In [47]:
# XGBoost
from xgboost.sklearn import XGBClassifier
cls=XGBClassifier(max_depth=10,min_samples_split=10,min_samples_leaf=5,max_features=0.8)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

0.9923297616956497

In [10]:
# lightGBM
from lightgbm import LGBMClassifier
cls=LGBMClassifier(boosting_type='gbdt', n_estimators=500, max_depth=-1,min_child_samples=10)

cls.fit(train_X,train_y)
y_pred=cls.predict_proba(val_X)[:,1]
y_pred = [int(round(r)) for r in y_pred]
metrics.f1_score(val_y,y_pred)

1.0

In [11]:
# 最终lightGBM在val上取得了1.0的f1，使用该分类器执行预测
# 获得线上1.0的成绩
test_data = pd.read_csv( "data_test.csv", sep=',', names=['f1', 'f2','f3','f4','f5','f6']).as_matrix()
result = open("result.csv", "w+", encoding="UTF-8")

res = cls.predict(test_data)

for i,r in enumerate(res):
    result.write("{},{}\n".format(i+1,int(round(r)) ))
result.close()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
